In [1]:
from datasets import load_dataset

ds = load_dataset("openai/gsm8k", "main")
ds

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 7473
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 1319
    })
})

In [3]:
raw_train_dataset = ds["train"]
raw_train_dataset[1]

{'question': 'Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?',
 'answer': 'Weng earns 12/60 = $<<12/60=0.2>>0.2 per minute.\nWorking 50 minutes, she earned 0.2 x 50 = $<<0.2*50=10>>10.\n#### 10'}

In [3]:
raw_train_dataset.features

{'question': Value(dtype='string', id=None),
 'answer': Value(dtype='string', id=None)}

In [4]:
from datasets import load_dataset

ds = load_dataset("openai/gsm8k", "socratic")

train-00000-of-00001.parquet:   0%|          | 0.00/2.68M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/487k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [5]:
raw_train_dataset = ds["train"]
raw_train_dataset[0]

{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
 'answer': 'How many clips did Natalia sell in May? ** Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nHow many clips did Natalia sell altogether in April and May? ** Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'}

In [1]:
import torch
from accelerate import Accelerator
from transformers import AutoModelForCausalLM, AutoTokenizer, get_scheduler, DataCollatorWithPadding
from torch.optim import AdamW
from datasets import load_dataset
from torch.utils.data import DataLoader
from accelerate import Accelerator
from tqdm.auto import tqdm
import evaluate

c:\Users\hans\.conda\envs\llm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
raw_datasets = load_dataset("openai/gsm8k", "main")
checkpoint = "meta-llama/Llama-3.1-8B"

tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.chat_template = (
    "{{ bos_token }}"
    "{% for message in messages %}"
    "{% if message['role'] == 'user' %}"
    "{{ 'User: ' + message['content'] }}"
    "{% elif message['role'] == 'assistant' %}"
    "{{ 'Assistant: ' + message['content'] }}"
    "{% endif %}"
    "{{ eos_token }}"
)
tokenizer.chat_template = chat_template
tokenizer.chat_template

"{% for message in messages %}{% if message['role'] == 'user' %}[USER]: {{ message['content'] }}{% elif message['role'] == 'assistant' %}[ASSISTANT]: {{message['content']}}{% endif %}{% endfor %}"

In [11]:
def tokenize_function(samples):
    conversations = []

    for question, answer in zip(samples['question'], samples['answer']):
        messages = [
            {"role": "user", "content": question},
            {"role": "assistant", "content": answer}
        ]
        conversation = tokenizer.apply_chat_template(
            messages, 
            tokenize=False,
            template=chat_template    
        )
        conversations.append(conversation)

    tokenized_output = tokenizer(
        conversations, 
        padding=True, 
        truncation=True,
        max_length=1024,
        return_tensors="pt"
    )

    return tokenized_output

print(raw_datasets["train"].column_names)

['question', 'answer']


In [12]:
# Correctly map in batch mode
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True, remove_columns=raw_datasets["train"].column_names)

Map: 100%|██████████| 1319/1319 [00:00<00:00, 3879.46 examples/s]


In [13]:
import evaluate

metric = evaluate.load("openai/gsm8k", "main")
model.eval()
for batch in eval_dl:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

print(metric.compute())

FileNotFoundError: Couldn't find a module script at d:\LLMResearch\MoBA\examples\openai\gsm8k\gsm8k.py. Module 'openai/gsm8k' doesn't exist on the Hugging Face Hub either.

In [14]:
evaluate.load??

Signature:
evaluate.load(
    path: str,
    config_name: Optional[str] = None,
    module_type: Optional[str] = None,
    process_id: int = 0,
    num_process: int = 1,
    cache_dir: Optional[str] = None,
    experiment_id: Optional[str] = None,
    keep_in_memory: bool = False,
    download_config: Optional[datasets.download.download_config.DownloadConfig] = None,
    download_mode: Optional[datasets.download.download_manager.DownloadMode] = None,
    revision: Union[str, datasets.utils.version.Version, NoneType] = None,
    **init_kwargs,
) -> evaluate.module.EvaluationModule
Source:   
def load(
    path: str,
    config_name: Optional[str] = None,
    module_type: Optional[str] = None,
    process_id: int = 0,
    num_process: int = 1,
    cache_dir: Optional[str] = None,
    experiment_id: Optional[str] = None,
    keep_in_memory: bool = False,
    download_config: Optional[DownloadConfig] = None,
    download_mode: Optional[DownloadMode] = None,
    revision: Optional[Union[str